# Analyzing NASA Planetary Exploration Budgets in SQL

For much of the last 60 years, NASA has been at the forefront of exploring our solar system. Budgeting is crucial for expensive science projects, especially those funded with public money.

I'll be using a cleaned-up version of the public dataset provided by [The Planetary Society](https://www.planetary.org/space-policy/planetary-exploration-budget-dataset). Accessing real financial information from NASA, a federal agency, is a rare opportunity.

## Total Cost of All Planetary Missions Over All Time

A good starting point for any budgetary analysis is determining the total expenditure.

For this analysis, we'll utilize the `mission_budgets` table, which contains the cost details of various aspects of missions over different fiscal years. The table has five columns:

- `mission`: The name of the mission.
- `fiscal_year`: The year for accounting purposes.
- `cost_type`: Fine-grained aspect of the project, e.g., "Spacecraft".
- `cost_group`: Broader aspect of the project, e.g., "Development/Implementation".
- `cost_MUSD`: Cost in million US dollars.

I'll now calculate the total cost of all missions over all time, labeled as `total_cost_MUSD`.

In [23]:
-- Total cost of all missions over all time
SELECT SUM("cost_MUSD") AS total_cost_MUSD
FROM mission_budgets;

,total_cost_musd
0,41406.554506


## Inflation Adjustment

Due to inflation, costs from the 1960s are not directly comparable to those in the 2020s. To accurately compare costs across different time periods, we need to adjust for inflation.

The `inflation` table provides correction factors for inflation adjustments. Each row in this table represents an inflation adjustment for a specific time period relative to today. It contains two columns:

- `fiscal_year`: The year for accounting purposes. Notably, in 1976, inflation was particularly high, so there are two values provided: "1976" represents the start of the year, while "1976TQ" represents the third quarter onwards.
- `inflation_adjustment`: This factor should be multiplied by currency values from the past to convert them into current currency values.

In [24]:
-- Calculate the total cost of all missions over all time, adjusted for inflation
SELECT SUM("cost_MUSD" * inflation_adjustment) AS adjusted_total_cost_MUSD
FROM public.mission_budgets
LEFT JOIN inflation
USING (fiscal_year);

,adjusted_total_cost_musd
0,80992.593983


## Identifying the Most Expensive Mission

While grand missions often capture headlines, questions about their costs inevitably arise. Understanding the expense of each project is vital, especially for budget-conscious decision-makers seeking potential cost savings.

Determining the most expensive mission is a critical task for anyone overseeing a budget.

In [25]:
-- Get the mission with the highest total cost
SELECT mission, 
       SUM("cost_MUSD" * inflation_adjustment) AS adjusted_total_cost_MUSD
FROM public.mission_budgets
LEFT JOIN inflation
USING (fiscal_year)
GROUP BY mission 
ORDER BY SUM("cost_MUSD" * inflation_adjustment) DESC
LIMIT 1;

,mission,adjusted_total_cost_musd
0,Viking,7208.541041


## Annual Expenditure Analysis

Examining annual expenditure provides valuable insights into budget trends. NASA's budgetary allocations have been subject to the ebbs and flows of the US economy and the varying levels of political support for planetary exploration over the decades.

Understanding the historical context of budgetary allocations is essential for gauging current financial perspectives. It allows stakeholders to assess their current budget in relation to past funding levels, providing valuable context for decision-making.

In [26]:
-- Calculate the inflation adjusted total cost per year
SELECT fiscal_year, 
       SUM("cost_MUSD" * inflation_adjustment) AS adjusted_total_cost_MUSD
FROM public.mission_budgets
LEFT JOIN inflation
USING (fiscal_year)
GROUP BY fiscal_year
ORDER BY fiscal_year

,fiscal_year,adjusted_total_cost_musd
0,1960,261.139746
1,1961,674.849760
2,1962,1624.488450
3,1963,2268.368675
4,1964,2412.935820
...,...,...
72,2031,149.605633
73,2032,149.590728
74,2033,149.593212
75,2034,128.050000


In [27]:
# Draw a bar plot of total cost vs. year
import plotly.express as px

px.bar(
 total_cost_by_year,
    x='fiscal_year',
    y='adjusted_total_cost_musd'
)


![Screenshot 2024-04-28 at 6.23.42 PM](Screenshot%202024-04-28%20at%206.23.42%20PM.png)


Our exploration budget data spans from 1960 to 2035, reflecting fluctuations influenced by political shifts. Some administrations prioritize space exploration more than others, resulting in peaks and valleys in funding. Notably, enthusiasm peaked before and after the moon landing.

However, there was a significant decrease in budget in 1976, attributed to high inflation in the USA. Looking ahead, there's a prudent allocation of funds for upcoming years, likely aimed at avoiding overburdening public finances.

## Budget Allocation by Destination

The 1960s witnessed the iconic "Space Race" between the United States and Russia, culminating in the historic moon landing. Since then, exploration efforts have extended beyond the moon to include other celestial bodies such as planets, asteroids, and comets.

To conduct an analysis of spending by destination, we'll utilize the `mission_details` table. Each row represents a distinct mission and contains four key columns:

- `mission`: The mission's name.
- `mission_full_name`: The formal name of the mission.
- `destination`: The celestial body or location within the solar system that the mission is designed to explore.
- `program`: The name of the NASA program to which the mission belongs.

In [28]:
-- Calculate the inflation adjusted total cost per destination
SELECT destination, 
       SUM("cost_MUSD" * inflation_adjustment) AS adjusted_total_cost_MUSD
FROM mission_budgets
LEFT JOIN inflation
USING (fiscal_year)
LEFT JOIN mission_details 
USING (mission) 
GROUP BY destination
ORDER BY adjusted_total_cost_MUSD DESC;

,destination,adjusted_total_cost_musd
0,Mars,31725.348594
1,Outer Planets,21827.564786
2,The Moon,12982.887263
3,Small Bodies,7365.860162
4,Venus,4796.703011
5,Mercury,1519.605957
6,Earth-Sun L1,427.220100
7,The Sun,347.404110


## Analysis: Changes in Spending by Destination Over Time

To gain insights into budgetary trends, we aim to understand how spending on different destinations has evolved over time. By examining historical data, we can uncover patterns and shifts in budget allocations across various celestial bodies within the solar system.

In [29]:
-- Calculate the inflation adjusted total cost per year per destination
SELECT fiscal_year,
       destination,
       SUM("cost_MUSD" * inflation_adjustment) AS adjusted_total_cost_MUSD
FROM mission_budgets
LEFT JOIN inflation
USING (fiscal_year)
LEFT JOIN mission_details 
USING (mission) 
GROUP BY fiscal_year, destination
ORDER BY fiscal_year, destination

,fiscal_year,destination,adjusted_total_cost_musd
0,1960,The Moon,261.139746
1,1961,The Moon,674.849760
2,1962,The Moon,1248.174750
3,1962,Venus,376.313700
4,1963,Mars,382.522200
...,...,...,...
277,2032,Small Bodies,21.540728
278,2033,Outer Planets,128.050000
279,2033,Small Bodies,21.543212
280,2034,Outer Planets,128.050000


In [31]:
fig = px.bar(total_cost_by_date_and_destination, x="fiscal_year", y="adjusted_total_cost_musd", color="destination")
fig.update_layout(xaxis={"categoryorder": "category ascending"})
fig.show()

![Screenshot 2024-04-28 at 7.07.39 PM](Screenshot%202024-04-28%20at%207.07.39%20PM.png)


In this visualization, three main colors stand out: dark blue representing the moon, light blue for Mars, and yellow for the outer planets.

Regarding the moon, the dominance of the space race is evident during the 1960s. However, post-space race, we observe a decline in interest in lunar exploration, with a subsequent shift towards Mars and then the outer planets. Notably, investigation into small bodies didn't commence until the mid-90s.
